
# Процесс обучения LLM

## Предобучение (pre-training)

Pre-training --- это первый и наиболее ресурсоёмкий этап обучения больших языковых моделей (LLM). В других областях машинного обучения, таких как компьютерное зрение, предобученные модели широко используются. Например, модель может предварительно обучаться на задаче классификации изображений (например, ImageNet), после чего её адаптируют к конкретной задаче путём замены выходного слоя и дообучения.

В NLP pre-training происходит на масштабных текстовых корпусах в режиме без учителя (unsupervised learning). LLM обучается на текстах из интернета, книг, научных статей, репозиториев с кодом и других источников. В процессе предобучения модель осваивает языковые структуры и накапливает знания о мире. В отличие от обучения на специализированных наборах данных, pre-training позволяет модели формировать универсальные представления о языке, применимые в широком спектре задач.

Для предобучения языковых моделей используются следующие задачи:
1. **Masked Language Modeling (MLM)**: В этом подходе (используемом, например, в BERT) часть слов в предложении скрывается, и модель должна восстановить их на основе контекста.    
2. **Causal Language Modeling (CLM)**: Этот метод (используемый в GPT) обучает модель предсказывать следующий токен на основе предыдущих.
3. **Next Sentence Prediction (NSP)**: В некоторых моделях (например, BERT) используется задача предсказания, идёт ли одно предложение после другого.

Unsupervised learning в направлен на восстановление вероятностного распределения данных. Поэтому в результате pre-training'а мы получаем вероятностную модель языка $p_\theta$ . После предобучения LLM можно адаптировать к различным задачам путём дополнительного обучения (fine-tuning). Далее речь пойдет про адаптацию LLM к реальным задачам.

## Alignment 

На этапе предобучения была получена модель, которая "содержит в себе знания о мире". Мы хотим научиться использовать данную модель для решения "продуктовых" задач.

Особенность языковых моделей заключается в том, что их можно использовать для решения как дискриминативных, так и генеративных задач. Дообучение под дискриминативные задачи не сильно отличается от дообучения моделей в других областях DL. Куда интереснее обстоят дела с генеративными задачами: мы хотим научиться генерировать "хорошие" объекты. 

Рассмотрим задачу **instruction following**: языковая модель должна отвечать на вопросы пользователя и "выполнять его поручения". Например, пользователь может попросить найти ошибку в коде и модель должна будет описать эту ошибку. На рисунке показан пример ответа базовой и дообученной моделей. Базовая модель "считает", что запрос пользователя является куском теста, поэтому модель генерирует вопрос с выбором ответа. Эта цепочка токенов имеет высокое правдоподобие, но это совсем не то, что хотел бы получить пользователь в ответ на свой вопрос. А вот ответ дообученной модели уже будет похож на то, чего хотел пользователь. Так как же добиться такого поведения?

![](images/Pasted%20image%2020250330201858.png)
### Supervised fine-tune (SFT)

Мы хотим, чтобы у "хороших" ответов $\textbf{ans}$ на запрос $\textbf{prompt}$ была высокая вероятность $p_\theta(\textbf{ans} \mid \textbf{prompt})$. Соберем датасет запросов и "эталонных" ответов на них $\{(\textbf{ans}_i, \, \textbf{prompt}_i)\}_{i=1}^l$ и будем решать задачу:
$$
\sum_i \log p_\theta (\textbf{ans}_i \mid \textbf{prompt}_i) \to\max_\theta
$$

У данного подхода есть ряд проблем:
1. Дорого размечать данные
2. Сдвиг распределения: на обучении предсказываем следующий токен по "эталонным" предыдущим. На инференсе предсказываем следующий токен по сгенерированным предыдущим
3. Решаем не ту задачу, которую хотим

**Важное замечание**: мы максимизируем $\log p_\theta (\textbf{ans}_i \mid \textbf{prompt}_i)$, а не $\log p_\theta (\textbf{ans}_i, \textbf{prompt}_i)$. Поэтому необходимо "замаскировать" промпт при подсчете градиента. Градиент должен течь только через ответ модели. Подробнее про это будет рассказано на семинаре.

### RLHF

![](images/Pasted%20image%2020250330190038.png)

Но основной проблемой является то, что постановка задачи не соответствует тому, что мы на самом деле хотим. Постановка задачи звучит так: "модель должна генерировать вот это, вот это и это. А всё остальное генерировать не должна". Но на вопрос пользователя можно ответить сильно по разному: можно использовать синонимы, иногда переставлять слова и т.д. Поэтому мы хотели бы решать задачу, которая формулируется так: "модель должна генерировать то, что "порадует" пользователя".

Предположим, что мы уже умеем оценивать качество ответа $\textbf{ans}$ на вопрос $\textbf{prompt}$. Пусть $Q(\textbf{ans}, \, \textbf{prompt})$ --- некоторая численная характеристика качества ответа, тогда мы хотим научиться решать задачу

$$
\underset{\substack{\textbf{prompt} \\ \textbf{ans} \sim p_\theta(\textbf{ans} \mid \textbf{prompt})}}{\mathbb{E}} \; Q(\textbf{ans}, \, \textbf{prompt}) \to \max_\theta \tag{1}
$$


Добавим к задаче $(1)$ регуляризацию $\texttt{KL}(p_\theta(\textbf{ans} \mid \textbf{prompt}) \| p_0(\textbf{ans} \mid \textbf{prompt}))$. Данная регуляризация не позволит слишком сильно отойти от исходного чекпоинта $p_0$. В качестве $p_0$ обычно берут чекпоинт модели после `SFT`-стадии. 

После добавления регуляризации задача принимает следующий вид:
$$
\underset{\substack{\textbf{prompt} \\ \textbf{ans} \sim p_\theta(\textbf{ans} \mid \textbf{prompt})}}{\mathbb{E}} \; Q(\textbf{ans}, \, \textbf{prompt}) \; + \; \beta \, \texttt{KL}(p_\theta \| p_0)  \to \max_\theta \tag{2}
$$

**Напоминание**:
$$
\texttt{KL}(p \| q) = \mathbb{E}_{\xi \sim p} \log \frac{p(\xi)}{q(\xi)}
$$

Раньше мы не сталкивались с оптимизацией подобных функционалов. Раньше от параметров модели зависела функция, для которой считается математическое ожидание. В данной задаче от параметров модели зависит распределение, по которому считается математическое ожидание. Для удобства дальнейших рассуждений переобозначим переменные.

Раньше мы решали задачу вида $\mathbb{E}_{\xi \sim \pi} f_\theta(\xi) \to \underset{\theta}{\max}$ при помощи стохастического градиентного спуска:
* "Честный" градиент имеет вид $\nabla_\theta \mathbb{E}_{\xi \sim \pi} f_\theta(\xi) = \mathbb{E}_{\xi \sim \pi} \nabla_\theta f_\theta(\xi)$
* На каждом шаге заменяем "честный" градиент на его Монте-Карло оценку по батчу $\xi_1, \ldots, \xi_k \sim \pi$
* Так как стохастический градиент является несмещенной оценкой "честного", то при правильном выборе lr (условия ...) процесс оптимизации "сходится" к окрестности оптимума

Попытаемся переписать "честный" градиент $\nabla_\theta \mathbb{E}_{\xi \sim \pi_\theta} f(\xi)$ в виде математического ожидания от какой-то функции:
$$
\begin{align}
	& \mathbb{E}_{\xi \sim \pi_\theta} \; f(\xi) = \int f(\xi) \pi_\theta(\xi) \, d \xi \\
	& \nabla_\theta \mathbb{E}_{\xi \sim \pi_\theta} f(\xi) = \int \nabla_\theta \left( f(\xi) \pi_\theta(\xi) \right)  \, d \xi = \int f(\xi) \nabla_\theta \pi_\theta(\xi) \, d \xi = \\
	& = \int f(\xi) \frac{\nabla_\theta\pi_\theta(\xi)}{\pi_\theta(\xi)} \pi_\theta(\xi) \, d \xi = \left\{ \nabla_\theta \log \pi_\theta(\xi) = \frac{\nabla_\theta\pi_\theta(\xi)}{\pi_\theta(\xi)}  \right\} = \\
	& = \int f(\xi) \nabla_\theta \log \pi_\theta(\xi) \; \pi_\theta(\xi) \, d\xi = \mathbb{E}_{\xi \sim \pi_\theta} f(\xi) \nabla_\theta \log \pi_\theta(\xi)
\end{align}
$$

Таким образом, мы научились считать стохастический градиент для данного функционала:
1. Семплируем батч $\xi_1, \ldots, \xi_k \sim \pi_\theta$ 
2. Вычисляем $\frac{1}{k} \sum_{i=1}^k \nabla_\theta \log \pi_\theta(\xi_i) f(\xi_i)$ 

Мы научились оптимизировать функционал $(2)$ (приведенный выше алгоритм носит название `REINFORCE`). Теперь обсудим, как строить функцию $Q(\textbf{ans}, \, \textbf{prompt})$

#### RL pipeline, PPO (proximal policy optimization)

Построить для каждого ответа численную характеристику его качества --- задача нетривиальная. Попробуем решить более простую задачу: нам дана пара ответов на один и тот же вопрос; мы хотим предсказать, какой из ответов понравится пользователю больше. Построим вероятностную модель (модель Брэдли-Терри):
$$
\mathbb{P}(\textbf{ans}_1 \succ \textbf{ans}_2 \mid \textbf{prompt}) = \sigma \left( r_\theta(\textbf{ans}_1, \textbf{prompt}) - r_\theta(\textbf{ans}_2, \textbf{prompt}) \right)
$$
Пусть у нас есть датасет запросов $\{\textbf{prompt}_i \}_{i=1}^l$. Для каждого запроса сгенерируем нашей моделью варианты ответов $\textbf{ans}^i_1, \ldots, \textbf{ans}^i_k$. Для каждой пары ответов попросим разметчиков выбрать лучший. Получим датасет $\{(\textbf{prompt}, \textbf{ans}_1, \textbf{ans}_2, y)\}$, где $y = 1 \iff \textbf{ans}_1 \succ \textbf{ans}_2$. Можно заметить, что мы пришли к задаче бинарной классификации $y$ по признаковому описанию $(\textbf{prompt}, \textbf{ans}_1, \textbf{ans}_2)$. Для удобства, переобозначим $\textbf{ans}_w \succ \textbf{ans}_l$. Будем обучать модель $r_\theta$ на задачу:
$$
\frac{1}{N} \sum_{i=1}^N \log \sigma \left(r_\theta(\textbf{ans}_w, \textbf{prompt}) - r_\theta(\textbf{ans}_l, \textbf{prompt}) \right) \to \max_\theta 
$$

Итоговый pipeline будет иметь следующий вид:
1. Генерируем нашей моделью ответы на вопросы
2. Используем людей для разметки сгенерированных ответов
3. (До)обучаем reward модель на размеченных данных
4. Используем `PPO`-алгоритм для обучение нашей модели
5. Возвращаемся к шагу 1 с обновленным чекпоинтом модели

![](images/Pasted%20image%2020250330161432.png)
На шаге 4 присутствует ранее не обсуждавшийся алгоритм `PPO`. Алгоритм `REINFORCE`  позволяет решить поставленную задачу, но он сходится крайне нестабильно. Причины этого будут подробно обсуждаться в разделе по RL на ММРО, на нейробайесовских методах и в курсе по RL. Поэтому на практике алгоритм `REINFORCE` не используются, вместо него используют другие более стабильные методы. В частности, используют алгоритм `PPO`. Желающие могут ознакомиться со статьей [PPO paper](https://arxiv.org/pdf/1707.06347)

Данный подход позволяет получить существенный прирост качества ответов относительно SFT-версии модели. Но у подхода есть много существенных недостатков:
1. Необходимо хранить сразу 4 модели в памяти (две модели мы обсудили: обучаемая и reward, еще 2 остались за кадром)
2. На каждом шаге алгоритма мы семплируем ответы. Семплирование происходит в авторегрессионном режиме, поэтому это **очень медленно**
3. Обучение не стабильно

#### DPO (direct preference optimization)

Рассмотрим алгоритм, который не обладает недостатками `PPO`. 

У регуляризированной задачи $(2)$ существует явное решение $p_*$:
$$
p_* (a \mid p) = \frac{1}{Z(p)} p_0(a \mid p) \exp \left( \frac{1}{\beta} Q(a, p) \right)
$$
Где $Z$ --- нормировочная константа:
$$
Z(p) = \sum_{a} p_0(a \mid p) \exp \left( \frac{1}{\beta} Q(a, p) \right)
$$
Мы не можем вычислить $Z$, так как суммирование ведется по всем возможным ответам $a$. Поэтому явный вид $p_*$ на практике не используется. 

В статье [DPO paper](https://proceedings.neurips.cc/paper_files/paper/2023/file/a85b405ed65c6477a4fe8302b5e06ce7-Paper-Conference.pdf) был предложен трюк, позволяющий избавиться от отдельной reward модели. Выразим $Q(a, p)$ через $p_*$:
$$
Q(s, a) = \beta \log \frac{p_*(a \mid p)}{p_0(a \mid p)} + \beta \log Z(p)
$$
Подставим полученное выражение $Q$ в задачу и заменим оптимальную модель $p_*$ на обучаемую $p_\theta$:
$$
\sum_i \log \sigma \left( \beta \log \frac{p_\theta (\textbf{ans}_w^i \mid \textbf{prompt}_i)}{p_0 (\textbf{ans}_w^i \mid \textbf{prompt}_i)} - \beta \log \frac{p_\theta (\textbf{ans}_l^i \mid \textbf{prompt}_i)}{p_0 (\textbf{ans}_l^i \mid \textbf{prompt}_i)} \right) \to \max_\theta
$$

На данный момент, описанный подход является SoTA. Его основные плюсы:
1. Стабильнее сходится
2. Всего один гиперпараметр $\beta$
3. Нужно хранить 2 модели вместо 4
4. Не нужно семплировать ответы, поэтому каждый шаг оптимизации происходит значительно быстрее

## Pipeline обучения

<img src="images/Pasted image 20250330134257.png" width="700"/>

Итоговый pipeline для alignment'а содержит `SFT` и `RLHF` стадии. На этапе `SFT` модель учится генерировать приемлемые ответы: мы, опираясь на эталонные ответы, показываем как правильно отвечать на вопросы\запросы пользователя. На этапе `RLHF` мы "донастраиваем" распределение ответов так, чтобы оно "удовлетворяло" пользователя. 

Были попытки отказаться от `SFT`стадии: DeepSeek-R1 обучалась только на RLHF. При отказе от `SFT` мы сильно экономим на разметке данных: вместо дорогого написания ответов разметчикам требуется сравнивать пары ответов. Но позже DeepSeek отметили, что при добавлении даже небольшой `SFT`-стадии в начало дообучения качество возрастает. Поэтому, на данный момент пайплайн содержит обе стадии.

# Методы дообучения

### Full fine-tune

Если у нас не ограничены вычислительные ресурсы, то мы можем позволить себе дообучать всю модель целиком. Это наиболее стабильный подход, который позволяет достигать максимально возможного качества на downstream задачах.
### Дообучение части слоёв 

В глубоком обучении существует гипотеза, что "уровень абстракции признаков" растет с номером слоя. В первых слоях выучиваются какие-то общие признаки, специфичные для всего домена. В последних слоях могут выучиваться признаки, специфичные для решаемой задачи. Поэтому при дообучении модели под downstream задачу можно прокидывать градиенты только через пару последних слоев. 

### PEFT

Если у не хватает вычислительных ресурсов или данных даже для дообучения части слоев, то дообучать модель можно в малопараметрическом классе. Рассмотрим наиболее популярную технику Parameter Efficient Fine-Tuning'а.

#### LoRA
Существует гипотеза, что скрытые представления лежат в окрестности многообразия малой размерности. Поэтому добавку к весам $W = W_0 + \Delta W$ можно искать в классе матриц ранга не более $r$, где $r \ll d$. Для параметризации $\Delta W \in \mathbb{R}^{d \times d}$ можно использовать скелетное разложение $\Delta W = A B^\top$, где $A, B \in \mathbb{R}^{d \times r}$.   

<!-- ![[Pasted image 20250330215332.png]] -->
![](images/Pasted%20image%2020250330215332.png)

Посчитаем сложность умножения матрицы $W$ на вектор $x$:
1. $z_0 = W_0 x$, сложность $d^2$
2. $z_1 = B^\top x$, сложность $dr$
3. $z_2 = A z_1$, сложность $dr$
4. $Wx = z_0 + z_2$, сложность $d$
Таким образом, добавка $\Delta W$ не сильно усложняет прямой проход через LLM.

Вместо вычисления градиента $\nabla_W \mathcal{L} \in \mathbb{R}^{d \times d}$ будут вычисляться градиенты $\nabla_A \mathcal{L}, \nabla_B \mathcal{L} \in \mathbb{R}^{d \times r}$. Это позволяет уменьшить вычислительную сложность обратного прохода, также существенно уменьшаются затраты по памяти.

Одним из основных преимуществ данной стратегии является возможность "встроить" адаптер в исходные веса, честно вычислив $W = W_0 + AB^\top$. Поэтому процесс инференса (и его вычислительная сложность) не будет отличаться от процесса инференса исходной модели.
#### Альтернативы
Помимо LoRA существует огромное количество других подходов к PEFT. 
1. DoRA и прочие аддитивные методы
2. Prefix tuning
3. OFT\BOFT\GSOFT: $W = Q W_0$, где $Q$ --- ортогональная матрица из некоторого малопараметрического класса. 

# In-context learning

Одним из ключевых свойств современных LLM является способность к **In-Context Learning (ICL)** — обучению на лету без обновления параметров модели. Вместо традиционного fine-tuning, модель получает несколько примеров входных данных и ожидаемых ответов в самом промпте, после чего продолжает генерацию на основе этих примеров.

#### Few-Shot, One-Shot и Zero-Shot Learning
- **Zero-shot learning**: Модель решает задачу без предоставления примеров, полагаясь на знания, накопленные во время pre-training.
- **One-shot learning**: Модели даётся один пример, по которому она должна обобщить правила задачи.
- **Few-shot learning**: Модель получает несколько примеров (обычно 2-5), что позволяет ей лучше понять контекст и требуемый формат вывода.

### Chain-of-Thought (CoT)

Обычные LLM выполняют предсказания на основе вероятностных распределений слов, но иногда сложные задачи требуют разбиения на несколько логических шагов. **Chain-of-Thought (CoT)** — это метод, в котором модель поощряется к генерации пошагового объяснения перед получением ответа. Это особенно полезно в задачах, требующих математических вычислений или сложных рассуждений. Например:

**Обычное предсказание:**

> Вопрос: Если у Анны 3 яблока, а у Боба на 2 яблока больше, сколько всего яблок? Ответ: 5.

**Chain-of-Thought:**

> Вопрос: Если у Анны 3 яблока, а у Боба на 2 яблока больше, сколько всего яблок? Шаг 1: У Боба 3 + 2 = 5 яблок. Шаг 2: Всего 3 + 5 = 8 яблок. Ответ: 8.

CoT улучшает интерпретируемость модели и её способность к решению многозадачных логических задач.

# Ссылки:
1. Курс по LLM от ВК: лекция 3 ([видео](https://disk.yandex.ru/d/phkqmuuTDcXwAg/%D0%9B%D0%B5%D0%BA%D1%86%D0%B8%D1%8F%2B%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%202025-01-29T13-12-41Z.mp4), [слайды](https://drive.google.com/file/d/1CxMO_W1L6PAUcJoL1tJXh_RjOMOrlvST/view)), лекция 4 ([видео](https://disk.yandex.ru/d/phkqmuuTDcXwAg/%D0%9B%D0%B5%D0%BA%D1%86%D0%B8%D1%8F%2B%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%202025-02-05T13-11-26Z.mp4), [слайды](https://drive.google.com/file/d/1ghccfQpj71nrlSbPn09iYdWE2b5lpXoZ/view)) 
2. Yandex GPT week: Alignment ([лекция](https://www.youtube.com/live/f5JFXvX7FLE?si=Npx0wZHc32twPYcR&t=294), [семинар](https://m.youtube.com/live/dcN0BFa0OAI?si=fj91gQPSAS-m-IPx&t=115)) 
3. [PPO paper](https://arxiv.org/pdf/1707.06347)
4. [DPO paper](https://proceedings.neurips.cc/paper_files/paper/2023/file/a85b405ed65c6477a4fe8302b5e06ce7-Paper-Conference.pdf)
5. [Training language models to follow instructionswith human feedback](https://arxiv.org/pdf/2203.02155)
6. [LoRA](https://arxiv.org/pdf/2106.09685)
7. 